# 📞 Análisis de Patrones en Llamadas IVR con LLM Local

Este notebook implementa un sistema completo para:
1. Cargar y procesar datos de llamadas telefónicas self-service
2. Generar embeddings locales usando Ollama
3. Almacenar en ChromaDB (base vectorial local)
4. Detectar patrones de fallo y explicarlos con LLM

**Requisitos:**
- Ollama instalado y corriendo localmente
- Modelo descargado (ej: `ollama pull llama3.1` y `ollama pull nomic-embed-text`)

## 1. Instalación de Dependencias

In [ ]:
# Instalar dependencias necesarias
!pip install chromadb pandas ollama scikit-learn matplotlib seaborn --quiet

In [ ]:
import pandas as pd
import numpy as np
import chromadb
import ollama
import json
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías cargadas correctamente")

## 2. Configuración de Ollama y ChromaDB

In [ ]:
@dataclass
class Config:
    """Configuración centralizada del sistema"""
    # Modelos de Ollama
    EMBEDDING_MODEL: str = "nomic-embed-text"  # Modelo para embeddings
    LLM_MODEL: str = "llama3.1"                # Modelo para análisis/explicaciones
    
    # ChromaDB
    CHROMA_PERSIST_DIR: str = "./chroma_ivr_db"
    COLLECTION_NAME: str = "ivr_call_patterns"
    
    # Análisis
    TOP_K_SIMILAR: int = 5  # Número de casos similares a recuperar
    
config = Config()
print(f"📋 Configuración:")
print(f"   - Modelo embeddings: {config.EMBEDDING_MODEL}")
print(f"   - Modelo LLM: {config.LLM_MODEL}")
print(f"   - Directorio ChromaDB: {config.CHROMA_PERSIST_DIR}")

In [ ]:
# Verificar conexión con Ollama
def verify_ollama_connection():
    """Verifica que Ollama esté corriendo y los modelos disponibles"""
    try:
        models = ollama.list()
        model_names = [m['name'].split(':')[0] for m in models.get('models', [])]
        print("✅ Ollama conectado correctamente")
        print(f"   Modelos disponibles: {model_names}")
        
        # Verificar modelos requeridos
        required = [config.EMBEDDING_MODEL, config.LLM_MODEL]
        for model in required:
            if model not in model_names:
                print(f"⚠️  Modelo '{model}' no encontrado. Ejecuta: ollama pull {model}")
        
        return True
    except Exception as e:
        print(f"❌ Error conectando con Ollama: {e}")
        print("   Asegúrate de que Ollama esté corriendo: ollama serve")
        return False

verify_ollama_connection()

## 3. Generación de Dataset de Ejemplo

Si ya tienes tu CSV, salta esta celda y carga tu archivo directamente.

In [ ]:
def generate_sample_dataset(n_calls: int = 500) -> pd.DataFrame:
    """
    Genera un dataset de ejemplo simulando llamadas IVR.
    Reemplaza esto con tu CSV real.
    """
    np.random.seed(42)
    
    # Definir pasos del flujo IVR
    ivr_steps = [
        "welcome_message",
        "language_selection",
        "authentication",
        "menu_principal",
        "submenu_consultas",
        "ingreso_datos",
        "validacion_datos",
        "procesamiento",
        "confirmacion",
        "despedida"
    ]
    
    # Posibles errores por paso
    error_types = {
        "authentication": ["auth_timeout", "invalid_credentials", "account_locked", "biometric_fail"],
        "ingreso_datos": ["dtmf_timeout", "invalid_input", "speech_not_recognized", "too_many_retries"],
        "validacion_datos": ["data_mismatch", "expired_data", "system_unavailable"],
        "procesamiento": ["backend_timeout", "service_unavailable", "transaction_failed"],
        "menu_principal": ["no_input", "invalid_option", "confusion_detected"]
    }
    
    calls = []
    
    for i in range(n_calls):
        call_id = f"CALL_{i:05d}"
        
        # Determinar si la llamada será exitosa (60% éxito)
        is_success = np.random.random() < 0.6
        
        if is_success:
            # Llamada exitosa - completa todos los pasos
            steps_completed = ivr_steps.copy()
            step_results = {step: "success" for step in steps_completed}
            final_result = "success"
            failure_step = None
            failure_reason = None
            end_action = "completed"
        else:
            # Llamada fallida - falla en algún paso
            fail_step_idx = np.random.choice([2, 3, 4, 5, 6, 7], p=[0.25, 0.1, 0.1, 0.25, 0.15, 0.15])
            steps_completed = ivr_steps[:fail_step_idx + 1]
            
            step_results = {step: "success" for step in steps_completed[:-1]}
            failure_step = steps_completed[-1]
            
            # Asignar tipo de error
            if failure_step in error_types:
                failure_reason = np.random.choice(error_types[failure_step])
            else:
                failure_reason = "unknown_error"
            
            step_results[failure_step] = "error"
            final_result = "error"
            
            # Acción final
            end_action = np.random.choice(["hangup", "transfer_agent"], p=[0.4, 0.6])
        
        # Metadata adicional
        call_duration = np.random.randint(30, 300) if is_success else np.random.randint(15, 180)
        retries = 0 if is_success else np.random.randint(1, 4)
        
        calls.append({
            "call_id": call_id,
            "timestamp": pd.Timestamp.now() - pd.Timedelta(days=np.random.randint(0, 30)),
            "steps_completed": json.dumps(steps_completed),
            "step_results": json.dumps(step_results),
            "final_result": final_result,
            "failure_step": failure_step,
            "failure_reason": failure_reason,
            "end_action": end_action,
            "call_duration_seconds": call_duration,
            "retry_count": retries,
            "customer_segment": np.random.choice(["premium", "standard", "basic"]),
            "call_type": np.random.choice(["billing", "support", "sales", "account_info"]),
            "hour_of_day": np.random.randint(8, 22),
            "day_of_week": np.random.choice(["lunes", "martes", "miercoles", "jueves", "viernes"])
        })
    
    return pd.DataFrame(calls)

# Generar dataset de ejemplo
df = generate_sample_dataset(500)
print(f"📊 Dataset generado: {len(df)} llamadas")
print(f"   - Exitosas: {len(df[df['final_result'] == 'success'])}")
print(f"   - Fallidas: {len(df[df['final_result'] == 'error'])}")
df.head()

In [ ]:
# ⬇️ ALTERNATIVA: Cargar tu propio CSV
# Descomenta y ajusta según tu estructura de datos

# df = pd.read_csv("tu_archivo.csv")
# print(f"Dataset cargado: {len(df)} registros")
# print(f"Columnas: {df.columns.tolist()}")

## 4. Preprocesamiento y Creación de Representaciones Textuales

In [ ]:
class CallDataProcessor:
    """
    Procesa los datos de llamadas y genera representaciones textuales
    para crear embeddings significativos.
    """
    
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.processed_calls = []
    
    def create_call_narrative(self, row: pd.Series) -> str:
        """
        Convierte una fila del DataFrame en una narrativa textual
        que capture el contexto completo de la llamada.
        """
        # Parsear JSON si es necesario
        if isinstance(row['steps_completed'], str):
            steps = json.loads(row['steps_completed'])
        else:
            steps = row['steps_completed']
            
        if isinstance(row['step_results'], str):
            results = json.loads(row['step_results'])
        else:
            results = row['step_results']
        
        # Construir narrativa
        narrative_parts = [
            f"Llamada tipo: {row['call_type']}",
            f"Segmento cliente: {row['customer_segment']}",
            f"Día: {row['day_of_week']}, Hora: {row['hour_of_day']}:00",
            f"Duración: {row['call_duration_seconds']} segundos",
            "\nFlujo de pasos:"
        ]
        
        # Describir cada paso
        for i, step in enumerate(steps, 1):
            result = results.get(step, 'unknown')
            status = "✓" if result == "success" else "✗"
            narrative_parts.append(f"  {i}. {step}: {status} ({result})")
        
        # Resultado final
        narrative_parts.append(f"\nResultado final: {row['final_result'].upper()}")
        
        if row['final_result'] == 'error':
            narrative_parts.extend([
                f"Paso de fallo: {row['failure_step']}",
                f"Razón del fallo: {row['failure_reason']}",
                f"Acción final: {row['end_action']}",
                f"Intentos de reintento: {row['retry_count']}"
            ])
        
        return "\n".join(narrative_parts)
    
    def create_pattern_signature(self, row: pd.Series) -> str:
        """
        Crea una firma de patrón más concisa para búsqueda rápida.
        """
        if isinstance(row['steps_completed'], str):
            steps = json.loads(row['steps_completed'])
        else:
            steps = row['steps_completed']
        
        signature_parts = [
            f"result:{row['final_result']}",
            f"steps:{len(steps)}",
            f"type:{row['call_type']}",
            f"segment:{row['customer_segment']}"
        ]
        
        if row['final_result'] == 'error':
            signature_parts.extend([
                f"fail_step:{row['failure_step']}",
                f"fail_reason:{row['failure_reason']}",
                f"end_action:{row['end_action']}"
            ])
        
        return " | ".join(signature_parts)
    
    def process_all(self) -> List[Dict]:
        """
        Procesa todas las llamadas y retorna lista de documentos.
        """
        self.processed_calls = []
        
        for idx, row in self.df.iterrows():
            doc = {
                "id": row['call_id'],
                "narrative": self.create_call_narrative(row),
                "signature": self.create_pattern_signature(row),
                "metadata": {
                    "call_id": row['call_id'],
                    "final_result": row['final_result'],
                    "failure_step": row['failure_step'] if pd.notna(row['failure_step']) else None,
                    "failure_reason": row['failure_reason'] if pd.notna(row['failure_reason']) else None,
                    "call_type": row['call_type'],
                    "customer_segment": row['customer_segment'],
                    "end_action": row['end_action']
                }
            }
            self.processed_calls.append(doc)
        
        return self.processed_calls

# Procesar datos
processor = CallDataProcessor(df)
processed_calls = processor.process_all()

print(f"✅ {len(processed_calls)} llamadas procesadas")
print("\n📝 Ejemplo de narrativa generada:")
print("-" * 50)
# Mostrar ejemplo de llamada fallida
failed_example = next(c for c in processed_calls if c['metadata']['final_result'] == 'error')
print(failed_example['narrative'])

## 5. Generación de Embeddings con Ollama

In [ ]:
class OllamaEmbeddings:
    """
    Genera embeddings usando Ollama localmente.
    """
    
    def __init__(self, model: str = "nomic-embed-text"):
        self.model = model
    
    def embed_single(self, text: str) -> List[float]:
        """Genera embedding para un texto individual."""
        response = ollama.embeddings(
            model=self.model,
            prompt=text
        )
        return response['embedding']
    
    def embed_batch(self, texts: List[str], show_progress: bool = True) -> List[List[float]]:
        """Genera embeddings para múltiples textos."""
        embeddings = []
        total = len(texts)
        
        for i, text in enumerate(texts):
            emb = self.embed_single(text)
            embeddings.append(emb)
            
            if show_progress and (i + 1) % 50 == 0:
                print(f"   Procesados: {i + 1}/{total}")
        
        return embeddings

# Inicializar generador de embeddings
embedder = OllamaEmbeddings(model=config.EMBEDDING_MODEL)

# Test rápido
test_emb = embedder.embed_single("prueba de embedding")
print(f"✅ Embeddings funcionando - Dimensión: {len(test_emb)}")

## 6. Almacenamiento en ChromaDB

In [ ]:
class IVRVectorStore:
    """
    Gestiona el almacenamiento vectorial de llamadas IVR usando ChromaDB.
    """
    
    def __init__(self, persist_dir: str, collection_name: str, embedder: OllamaEmbeddings):
        self.embedder = embedder
        
        # Inicializar ChromaDB con persistencia
        self.client = chromadb.PersistentClient(path=persist_dir)
        
        # Crear o recuperar colección
        self.collection = self.client.get_or_create_collection(
            name=collection_name,
            metadata={"description": "IVR call patterns for failure analysis"}
        )
        
        print(f"✅ ChromaDB inicializado")
        print(f"   Colección: {collection_name}")
        print(f"   Documentos existentes: {self.collection.count()}")
    
    def add_calls(self, processed_calls: List[Dict], batch_size: int = 100):
        """
        Añade llamadas procesadas a la base vectorial.
        """
        print(f"\n📥 Indexando {len(processed_calls)} llamadas...")
        
        # Preparar datos para ChromaDB
        ids = []
        documents = []
        metadatas = []
        
        for call in processed_calls:
            ids.append(call['id'])
            # Combinamos narrativa y firma para el documento
            documents.append(f"{call['narrative']}\n\nPatrón: {call['signature']}")
            metadatas.append(call['metadata'])
        
        # Generar embeddings
        print("   Generando embeddings...")
        embeddings = self.embedder.embed_batch(documents)
        
        # Insertar en batches
        for i in range(0, len(ids), batch_size):
            end_idx = min(i + batch_size, len(ids))
            
            self.collection.add(
                ids=ids[i:end_idx],
                documents=documents[i:end_idx],
                embeddings=embeddings[i:end_idx],
                metadatas=metadatas[i:end_idx]
            )
        
        print(f"✅ {len(ids)} llamadas indexadas correctamente")
        print(f"   Total en colección: {self.collection.count()}")
    
    def search_similar(self, query_text: str, n_results: int = 5, 
                       filter_result: Optional[str] = None) -> Dict:
        """
        Busca llamadas similares a una consulta.
        """
        # Generar embedding de la consulta
        query_embedding = self.embedder.embed_single(query_text)
        
        # Construir filtro opcional
        where_filter = None
        if filter_result:
            where_filter = {"final_result": filter_result}
        
        # Buscar
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results,
            where=where_filter,
            include=["documents", "metadatas", "distances"]
        )
        
        return results
    
    def get_failure_patterns(self) -> Dict:
        """
        Obtiene estadísticas de patrones de fallo.
        """
        # Obtener todos los documentos de error
        results = self.collection.get(
            where={"final_result": "error"},
            include=["metadatas"]
        )
        
        # Analizar patrones
        failure_steps = Counter()
        failure_reasons = Counter()
        end_actions = Counter()
        
        for meta in results['metadatas']:
            if meta.get('failure_step'):
                failure_steps[meta['failure_step']] += 1
            if meta.get('failure_reason'):
                failure_reasons[meta['failure_reason']] += 1
            if meta.get('end_action'):
                end_actions[meta['end_action']] += 1
        
        return {
            "total_failures": len(results['metadatas']),
            "by_step": dict(failure_steps.most_common()),
            "by_reason": dict(failure_reasons.most_common()),
            "by_end_action": dict(end_actions.most_common())
        }
    
    def clear_collection(self):
        """Limpia la colección (útil para re-entrenar)."""
        self.client.delete_collection(self.collection.name)
        self.collection = self.client.create_collection(
            name=self.collection.name,
            metadata={"description": "IVR call patterns for failure analysis"}
        )
        print("🗑️ Colección limpiada")

In [ ]:
# Inicializar vector store
vector_store = IVRVectorStore(
    persist_dir=config.CHROMA_PERSIST_DIR,
    collection_name=config.COLLECTION_NAME,
    embedder=embedder
)

# Limpiar si queremos empezar fresco (opcional)
# vector_store.clear_collection()

# Indexar llamadas solo si la colección está vacía
if vector_store.collection.count() == 0:
    vector_store.add_calls(processed_calls)
else:
    print(f"ℹ️ Ya hay {vector_store.collection.count()} documentos indexados")

## 7. Motor de Análisis con LLM

In [ ]:
class IVRPatternAnalyzer:
    """
    Analiza patrones de fallo usando RAG + LLM local.
    """
    
    def __init__(self, vector_store: IVRVectorStore, llm_model: str):
        self.vector_store = vector_store
        self.llm_model = llm_model
    
    def _build_analysis_prompt(self, new_call: Dict, similar_cases: Dict) -> str:
        """
        Construye el prompt para el análisis con contexto de casos similares.
        """
        # Formatear casos similares
        similar_context = ""
        if similar_cases['documents'] and similar_cases['documents'][0]:
            for i, (doc, meta, dist) in enumerate(zip(
                similar_cases['documents'][0],
                similar_cases['metadatas'][0],
                similar_cases['distances'][0]
            ), 1):
                similar_context += f"\n--- Caso Similar #{i} (similitud: {1-dist:.2f}) ---\n"
                similar_context += f"Resultado: {meta.get('final_result', 'N/A')}\n"
                if meta.get('failure_step'):
                    similar_context += f"Paso de fallo: {meta['failure_step']}\n"
                    similar_context += f"Razón: {meta.get('failure_reason', 'N/A')}\n"
                similar_context += "\n"
        
        prompt = f"""Eres un experto en análisis de sistemas IVR (Interactive Voice Response) y patrones de fallo en llamadas telefónicas self-service.

Tu tarea es analizar una nueva llamada y explicar la causa probable del fallo basándote en:
1. Los datos de la llamada actual
2. Patrones encontrados en casos similares históricos

## NUEVA LLAMADA A ANALIZAR:
{new_call['narrative']}

## CASOS HISTÓRICOS SIMILARES:
{similar_context if similar_context else 'No se encontraron casos similares.'}

## INSTRUCCIONES:
Proporciona un análisis estructurado que incluya:

1. **DIAGNÓSTICO**: Explica qué pasó en esta llamada y por qué falló.

2. **PATRÓN IDENTIFICADO**: Describe si este fallo sigue un patrón común basado en los casos similares.

3. **CAUSA RAÍZ PROBABLE**: Identifica la causa más probable del fallo.

4. **RECOMENDACIONES**: Sugiere mejoras específicas para prevenir este tipo de fallos.

5. **NIVEL DE CONFIANZA**: Indica qué tan seguro estás del diagnóstico (Alto/Medio/Bajo) y por qué.

Responde en español y sé específico con los nombres de pasos y errores."""
        
        return prompt
    
    def analyze_call(self, call_data: Dict, find_similar: bool = True) -> str:
        """
        Analiza una llamada y genera explicación del fallo.
        """
        # Procesar la nueva llamada
        processor = CallDataProcessor(pd.DataFrame([call_data]))
        processed = processor.process_all()[0]
        
        # Buscar casos similares
        similar_cases = {'documents': [[]], 'metadatas': [[]], 'distances': [[]]}
        if find_similar:
            similar_cases = self.vector_store.search_similar(
                processed['narrative'],
                n_results=config.TOP_K_SIMILAR,
                filter_result="error"  # Solo buscar entre fallos
            )
        
        # Construir prompt
        prompt = self._build_analysis_prompt(processed, similar_cases)
        
        # Generar análisis con LLM
        response = ollama.generate(
            model=self.llm_model,
            prompt=prompt,
            options={
                "temperature": 0.3,  # Más determinístico para análisis
                "num_predict": 1000
            }
        )
        
        return response['response']
    
    def get_pattern_summary(self) -> str:
        """
        Genera un resumen de patrones de fallo usando el LLM.
        """
        patterns = self.vector_store.get_failure_patterns()
        
        prompt = f"""Analiza las siguientes estadísticas de fallos en un sistema IVR y proporciona un resumen ejecutivo:

## ESTADÍSTICAS DE FALLOS:
- Total de llamadas fallidas: {patterns['total_failures']}

### Fallos por paso:
{json.dumps(patterns['by_step'], indent=2)}

### Fallos por razón:
{json.dumps(patterns['by_reason'], indent=2)}

### Acción final del usuario:
{json.dumps(patterns['by_end_action'], indent=2)}

Proporciona:
1. Los 3 problemas más críticos
2. Patrones preocupantes
3. Recomendaciones priorizadas de mejora

Responde en español de forma concisa y accionable."""
        
        response = ollama.generate(
            model=self.llm_model,
            prompt=prompt,
            options={"temperature": 0.3}
        )
        
        return response['response']

# Inicializar analizador
analyzer = IVRPatternAnalyzer(vector_store, config.LLM_MODEL)
print("✅ Analizador de patrones inicializado")

## 8. Uso del Sistema - Ejemplos Prácticos

In [ ]:
# Ver estadísticas de patrones
patterns = vector_store.get_failure_patterns()
print("📊 ESTADÍSTICAS DE PATRONES DE FALLO")
print("=" * 50)
print(f"\nTotal llamadas fallidas: {patterns['total_failures']}")
print(f"\nFallos por paso:")
for step, count in patterns['by_step'].items():
    print(f"   {step}: {count}")
print(f"\nFallos por razón:")
for reason, count in patterns['by_reason'].items():
    print(f"   {reason}: {count}")

In [ ]:
# Generar resumen ejecutivo con LLM
print("📋 RESUMEN EJECUTIVO DE PATRONES")
print("=" * 50)
summary = analyzer.get_pattern_summary()
print(summary)

### 8.1 Analizar una Nueva Llamada

In [ ]:
# Simular una nueva llamada fallida para analizar
nueva_llamada = {
    "call_id": "NEW_001",
    "timestamp": pd.Timestamp.now(),
    "steps_completed": json.dumps([
        "welcome_message",
        "language_selection",
        "authentication"
    ]),
    "step_results": json.dumps({
        "welcome_message": "success",
        "language_selection": "success",
        "authentication": "error"
    }),
    "final_result": "error",
    "failure_step": "authentication",
    "failure_reason": "auth_timeout",
    "end_action": "transfer_agent",
    "call_duration_seconds": 45,
    "retry_count": 2,
    "customer_segment": "premium",
    "call_type": "billing",
    "hour_of_day": 14,
    "day_of_week": "miercoles"
}

print("🔍 ANALIZANDO NUEVA LLAMADA")
print("=" * 50)
print(f"ID: {nueva_llamada['call_id']}")
print(f"Tipo: {nueva_llamada['call_type']}")
print(f"Fallo en: {nueva_llamada['failure_step']}")
print(f"Razón: {nueva_llamada['failure_reason']}")
print("\n" + "=" * 50)
print("ANÁLISIS DEL LLM:")
print("=" * 50 + "\n")

analisis = analyzer.analyze_call(nueva_llamada)
print(analisis)

### 8.2 Buscar Casos Similares

In [ ]:
# Buscar casos similares a un patrón específico
query = "llamada de billing que falla en autenticación con timeout"

print(f"🔎 Buscando casos similares a: '{query}'")
print("=" * 50)

resultados = vector_store.search_similar(
    query_text=query,
    n_results=3,
    filter_result="error"
)

for i, (doc, meta, dist) in enumerate(zip(
    resultados['documents'][0],
    resultados['metadatas'][0],
    resultados['distances'][0]
), 1):
    print(f"\n--- Resultado #{i} (distancia: {dist:.4f}) ---")
    print(f"Call ID: {meta['call_id']}")
    print(f"Paso de fallo: {meta.get('failure_step', 'N/A')}")
    print(f"Razón: {meta.get('failure_reason', 'N/A')}")
    print(f"Tipo: {meta.get('call_type', 'N/A')}")

## 9. Función Helper para Análisis Rápido

In [ ]:
def analizar_fallo_rapido(
    call_type: str,
    failure_step: str,
    failure_reason: str,
    customer_segment: str = "standard",
    end_action: str = "hangup",
    retries: int = 1
) -> str:
    """
    Función helper para análisis rápido de un caso de fallo.
    
    Ejemplo:
        resultado = analizar_fallo_rapido(
            call_type="billing",
            failure_step="authentication",
            failure_reason="biometric_fail",
            customer_segment="premium"
        )
    """
    # Construir pasos hasta el fallo
    all_steps = [
        "welcome_message",
        "language_selection",
        "authentication",
        "menu_principal",
        "submenu_consultas",
        "ingreso_datos",
        "validacion_datos",
        "procesamiento"
    ]
    
    # Encontrar índice del paso de fallo
    if failure_step in all_steps:
        fail_idx = all_steps.index(failure_step)
        steps_completed = all_steps[:fail_idx + 1]
    else:
        steps_completed = ["welcome_message", failure_step]
    
    # Construir resultados
    step_results = {step: "success" for step in steps_completed[:-1]}
    step_results[failure_step] = "error"
    
    llamada = {
        "call_id": "QUICK_ANALYSIS",
        "timestamp": pd.Timestamp.now(),
        "steps_completed": json.dumps(steps_completed),
        "step_results": json.dumps(step_results),
        "final_result": "error",
        "failure_step": failure_step,
        "failure_reason": failure_reason,
        "end_action": end_action,
        "call_duration_seconds": 60,
        "retry_count": retries,
        "customer_segment": customer_segment,
        "call_type": call_type,
        "hour_of_day": 12,
        "day_of_week": "miercoles"
    }
    
    return analyzer.analyze_call(llamada)

print("✅ Función analizar_fallo_rapido() disponible")

In [ ]:
# Ejemplo de uso de la función rápida
print("🚀 ANÁLISIS RÁPIDO")
print("=" * 50)

resultado = analizar_fallo_rapido(
    call_type="support",
    failure_step="ingreso_datos",
    failure_reason="speech_not_recognized",
    customer_segment="basic",
    end_action="transfer_agent",
    retries=3
)

print(resultado)

## 10. Exportar y Guardar Resultados

In [ ]:
# Guardar el dataset de ejemplo para referencia
df.to_csv("ivr_calls_sample.csv", index=False)
print("✅ Dataset guardado en 'ivr_calls_sample.csv'")

# Guardar patrones identificados
patterns = vector_store.get_failure_patterns()
with open("failure_patterns.json", "w") as f:
    json.dump(patterns, f, indent=2)
print("✅ Patrones guardados en 'failure_patterns.json'")

---

## 📚 Resumen de Uso

### Componentes principales:

1. **`CallDataProcessor`**: Convierte datos CSV en narrativas textuales
2. **`OllamaEmbeddings`**: Genera embeddings usando modelo local
3. **`IVRVectorStore`**: Almacena y busca en ChromaDB
4. **`IVRPatternAnalyzer`**: Analiza patrones con LLM

### Flujo típico:

```python
# 1. Cargar datos
df = pd.read_csv("tu_archivo.csv")

# 2. Procesar e indexar
processor = CallDataProcessor(df)
processed = processor.process_all()
vector_store.add_calls(processed)

# 3. Analizar nueva llamada
resultado = analyzer.analyze_call(nueva_llamada)

# 4. O usar función rápida
resultado = analizar_fallo_rapido(
    call_type="billing",
    failure_step="authentication",
    failure_reason="timeout"
)
```

### Modelos recomendados para Ollama:

- **Embeddings**: `nomic-embed-text` (rápido y efectivo)
- **LLM**: `llama3.1` o `mistral` (buenos para análisis en español)
- **Alternativa ligera**: `phi3` o `gemma2` si tienes recursos limitados